# GENERATING MACHINE LEARNING MODEL

In this kernel, we will generate a machine learning model decision tree. Some  data preparation tasks need to be performed first as follows:
1. Replace null values with median of that column.
2. Convert categorical variables to numerical variables.
3. Prepare training dan test data.

The kernel is based on Kaggle Tutorial: EDA & Machine Learning and Kaggle Tutorial: Your First Machine Learning Model by Hugo-Brown Anderson.

In this tutorial, we will generate a decision tree model of Titanic dataset to predict the survival of its passengers.

We will use the decision tree implementation from a machine learning library, Scikit Learn.

You may need to download and install some libraries e.g. sklearn, seaborn etc. You can use the pip instruction to install the required libraries.

Import data and check it out.

In [1]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Figures inline and set visualization style
%matplotlib inline
sns.set()

Read the dataset.

In [2]:
# Import data
df_train = pd.read_csv('data/train-1.csv')

In [3]:
df_train.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


## 1. First, we need to do some preprocessing on the dataset. 

Do some preprocessing to the dataset. We need to temporarily remove the target class, `Survived`, from the dataset.

But first, you'll store the target variable of the training data for safe keeping.

Let us see what we have in the dataset. We will print the first 3 rows of the data.

In [4]:
# Store target variable of training data in a safe place
survived_train = df_train.Survived

# Drop the Survived column
data = df_train.drop(['Survived'], axis=1)

Check out your new DataFrame data using the `info()` method.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871 entries, 0 to 870
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  871 non-null    int64  
 1   Pclass       871 non-null    int64  
 2   Name         871 non-null    object 
 3   Sex          871 non-null    object 
 4   Age          696 non-null    float64
 5   SibSp        871 non-null    int64  
 6   Parch        871 non-null    int64  
 7   Ticket       871 non-null    object 
 8   Fare         871 non-null    float64
 9   Cabin        199 non-null    object 
 10  Embarked     869 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 75.0+ KB


Some numerical variables have missing values, `Age` and `Fare`. Some categorical variables are also have missing values; `Cabin` and `Embarked`.

Now we will focus on fixing the numerical variables: you will impute or fill in the missing values for those columns, using the median of these variables where you know them. Impute is a process to remove null values in your dataset by replacing them with other values.

Note that in this case, you use the median because it's perfect for dealing with outliers. In other words, the median is useful to use when the distribution of data is skewed. Other ways to impute the missing values would be to use the mean, which you can find by adding all data points and dividing by the number of data points, or mode, which is the number that occurs the highest number of times.

In [6]:
# fill missing values with median column values
data.fillna(data.median(), inplace=True)

Now check out the modified data. You can see all the numerical variables are now consists of non-null numerical values.

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871 entries, 0 to 870
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  871 non-null    int64  
 1   Pclass       871 non-null    int64  
 2   Name         871 non-null    object 
 3   Sex          871 non-null    object 
 4   Age          871 non-null    float64
 5   SibSp        871 non-null    int64  
 6   Parch        871 non-null    int64  
 7   Ticket       871 non-null    object 
 8   Fare         871 non-null    float64
 9   Cabin        199 non-null    object 
 10  Embarked     869 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 75.0+ KB


Most machine learning models work input features that are numerical. Hence you want to encode your data with numbers, so you'll want to change 'object' dtypes variables to numbers. You can use the pandas function `.get_dummies()` to do so. The `.get_dummies()` allows you to create a new column for each of the options in categorical variables. 

In this example, we would like to convert the `Sex` attribute to numeric.

In [8]:
data = pd.get_dummies(data, columns=['Sex'], drop_first=True)

data.head()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
0,1,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,0
4,5,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,1


`.get_dummies()` allows you to create a new column for each of the options in 'Sex'. So it creates a new column for `female`, called `'Sex_female'`, and then a new column for `'Sex_male'`, which encodes whether that row was male or female.

Now, because you added the `drop_first` argument in the line of code above, you dropped `'Sex_female'` because, essentially, these new columns, `'Sex_female'` and `'Sex_male'`, encode the same information.

So all you have done is create a new column `'Sex_male'`, which has a 1 if that row is a male - and a 0 if that row is female.

Now, you'll select the columns `['Sex_male', 'Fare', 'Age','Pclass', 'SibSp']` from your DataFrame to build your first machine learning model:

In [9]:
# Select columns and view head
data = data[['Sex_male', 'Fare', 'Age','Pclass', 'SibSp']]
data.head()

,Sex_male,Fare,Age,Pclass,SibSp
0,1,7.2500,22.0,3,1
1,0,71.2833,38.0,1,1
2,0,7.9250,26.0,3,0
3,0,53.1000,35.0,1,1
4,1,8.0500,35.0,3,0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871 entries, 0 to 870
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sex_male  871 non-null    uint8  
 1   Fare      871 non-null    float64
 2   Age       871 non-null    float64
 3   Pclass    871 non-null    int64  
 4   SibSp     871 non-null    int64  
dtypes: float64(2), int64(2), uint8(1)
memory usage: 28.2 KB


All the entries are non-null now!

Now, you have got your data in a form to build first machine learning model.

## 2. Build a Decision Tree Classifier

What is a decision tree classifier? It is a tree that allows you to classify data points, which are also known as target variables, based on feature variables.

In this lab, we are using the sklearn tree library to develop our decision tree classifier. Documentation of this library can be found at https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html.

Before fitting a model to your data, we split it into training and test sets.

The training set will be used for the decision tree to learn the dataset. It will output a model or classifier.

The test data will be used to test how good the genefrated classifier is.

Split the dataset into train and test data. Lets set the test size to 0.2.

In [11]:
# create training and testing vars
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
train_label = survived_train[msk]
test = data[~msk]
test_label = survived_train[~msk]

print('Number of records in dataset: ', len(data))
print('Number of records in training set: ', len(train))
print('Number of frecords in test set: ', len(test))

Number of records in dataset:  871
Number of records in training set:  690
Number of frecords in test set:  181


You'll use scikit-learn, which requires your data as arrays, not DataFrames so transform them:

In [12]:
X = train.values
y = train_label.values

X_test = test.values
y_test = test_label.values

Now you get to build your decision tree classifier! First create such a model with `max_depth=3` and then fit it your data. Note that you name your model clf, which is short for "Classifier"

In [13]:
# Instatntiate model and fit to data
clf = tree.DecisionTreeClassifier(max_depth=3)
clf.fit(X, y)

DecisionTreeClassifier(max_depth=3)

The feature variables X is the first argument that you pass to the `.fit()` method, while the target variable, y, is the second argument.

The output tells you all that you need to know about your Decision Tree Classifier that you just built: as such, you see, for example, that the max depth is set at 3.

Now, you'll make predictions on your training set and test set.

In [ ]:
#Compute accuracy on the training set
train_accuracy = clf.score(X, y)

#Compute accuracy on the testing set
test_accuracy = clf.score(X_test, y_test)

print('Training accuracy: ', train_accuracy)
print('Test accuracy: ', test_accuracy)

In the above code, the test_accuracy stores the accuracy of the generated classifier. named clf, in predicting the survival of the passengers listed in the test set.

From the generated result, the training set produced an accuracy at 83.4% while test accuracy is at 80.9%. 

You may improve the accuracy by choosing the suitable `max_depth`. 

### What is `max_depth`?

The depth of the tree is known as a hyperparameter, which means a parameter you need to decide before you fit the model to the data. If you choose a larger max_depth, you'll get a more complex decision boundary.

    If your decision boundary is too complex, you can overfit to the data, which means that your model will be describing noise as well as signal.

    If your max_depth is too small, you might be underfitting the data, meaning that your model doesn't contain enough of the signal.

So how  do you identify the best `max_depth`?

One way is to hold out a test set from your training data. You can then fit the model to your training data, make predictions on your test set and see how well your prediction does on the test set. 

### Display the generated tree.

We can plot the tree with the `plot_tree` function.

You may need to install a graphviz library.

In [ ]:
#dot_data = StringIO('tree.dot')
from sklearn.tree import export_graphviz
import io
from graphviz import Source

from IPython.display import Image
from IPython.display import SVG
from IPython.display import display

export_graphviz(clf, out_file='tree.dot', filled=True, 
                rounded=True, special_characters=True, class_names=['0','1'])

# Convert to png
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

#graph = pydotplus.graph_from_dot_data(dot_data)
Image(filename='tree.png')

In this tutorial, you have:
1. Prepare the data in a form to build a machine learning model.
2. Built a machine learning model using a decision tree classifier.
3. Makes prediction based on the generated model.

# Ungraded assessment

## Predict if the unknown passengers are survived/ not survived.

These are ungraded questions, but your submission will be used as a proof of attendance. Different from the previous lab, submission that produced the best accuracy of prediction will be rewarded. You may form a group of two. I will select three winners, which will be determined based on the accuracy of the prediction, correctness of the code and the time that you need to submit your answer. All the submissions must be accompanied by the runnable code that produced the prediction.

Please use the test data provided, 'test-no-label.csv'. Your group are required to predict the 'Survived' label of each of the passenger listed in the test file.

In [14]:
# Import test data
df_test = pd.read_csv('data/test-no-label.csv')

In [15]:
data_test = df_test

In [16]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  20 non-null     int64  
 1   Pclass       20 non-null     int64  
 2   Name         20 non-null     object 
 3   Sex          20 non-null     object 
 4   Age          18 non-null     float64
 5   SibSp        20 non-null     int64  
 6   Parch        20 non-null     int64  
 7   Ticket       20 non-null     object 
 8   Fare         20 non-null     float64
 9   Cabin        5 non-null      object 
 10  Embarked     20 non-null     object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1.8+ KB


In [17]:
# fill missing values with median column values
data_test.fillna(data.median(), inplace=True)

In [18]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  20 non-null     int64  
 1   Pclass       20 non-null     int64  
 2   Name         20 non-null     object 
 3   Sex          20 non-null     object 
 4   Age          20 non-null     float64
 5   SibSp        20 non-null     int64  
 6   Parch        20 non-null     int64  
 7   Ticket       20 non-null     object 
 8   Fare         20 non-null     float64
 9   Cabin        5 non-null      object 
 10  Embarked     20 non-null     object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1.8+ KB


In [19]:
data_test = pd.get_dummies(data_test, columns=['Sex'], drop_first=True)

In [20]:
data_test.head()

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
0,872,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",47.0,1,1,11751,52.5542,D35,S,0
1,873,1,"Carlsson, Mr. Frans Olof",33.0,0,0,695,5.0000,B51 B53 B55,S,1
2,874,3,"Vander Cruyssen, Mr. Victor",47.0,0,0,345765,9.0000,NaN,S,1
3,875,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",28.0,1,0,P/PP 3381,24.0000,NaN,C,0
4,876,3,"Najib, Miss. Adele Kiamie ""Jane""",15.0,0,0,2667,7.2250,NaN,C,0


In [21]:
data_test = data_test[['Sex_male', 'Fare', 'Age','Pclass', 'SibSp']]

In [22]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sex_male  20 non-null     uint8  
 1   Fare      20 non-null     float64
 2   Age       20 non-null     float64
 3   Pclass    20 non-null     int64  
 4   SibSp     20 non-null     int64  
dtypes: float64(2), int64(2), uint8(1)
memory usage: 788.0 bytes


In [27]:
#convert to array
x_test = data_test.values

In [28]:
#make prediction
y_predict_clf = clf.predict(x_test)

In [33]:
y_predict_clf[:20]

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
      dtype=int64)

In [37]:
updated_data = df_test

#update passenger data with predicted survival
updated_data['Survived'] = y_predict_clf

In [39]:
#view updated data
updated_data.head(20)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,872,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S,1
1,873,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S,0
2,874,3,"Vander Cruyssen, Mr. Victor",male,47.0,0,0,345765,9.0000,NaN,S,0
3,875,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.0000,NaN,C,1
4,876,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C,1
5,877,3,"Gustafsson, Mr. Alfred Ossian",male,20.0,0,0,7534,9.8458,NaN,S,0
6,878,3,"Petroff, Mr. Nedelio",male,19.0,0,0,349212,7.8958,NaN,S,0
7,879,3,"Laleff, Mr. Kristo",male,28.0,0,0,349217,7.8958,NaN,S,0
8,880,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,1
9,881,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S,1
